# Resolução de problemas por meio de busca



Algoritmos de busca desempenham um papel fundamental na área de Inteligência Artificial, especialmente quando se trata da resolução de problemas baseados em objetivos. Esses algoritmos fornecem uma abordagem sistemática e eficiente para encontrar soluções em um espaço de busca definido. Nesse contexto, exploraremos os algoritmos de busca sem informação, como a busca em largura, a busca em profundidade limitada e a busca em aprofundamento iterativo em profundidade.

Para essa análise, utilizaremos problema do 8 puzzle, que é um quebra-cabeça deslizante em um tabuleiro 3x3 com oito peças numeradas de 1 a 8 e um espaço vazio. O objetivo é chegar a uma configuração final a partir de uma configuração inicial, deslizando as peças no tabuleiro. É um problema clássico usado para estudar algoritmos de busca em Inteligência Artificial.

In [1]:
from typing import Deque, Any
from collections import deque
import copy
import time
import psutil

In [2]:
class Queue():
  def __init__(self, maxLen = None) -> None:
    #define uma propriedade privada que é um objeto vazio do tipo Deque
    #maxLen define o tamanho maximo da fila, 
    #se não for dado, pode crescer indefinidamente
    self.__items: Deque[Any] = deque(maxlen = maxLen)
  
  def enqueue(self, *items: Any) -> None:
    for item in items:
      self.__items.append(item)
  
  def dequeue(self) -> Any:
    if not self:
      raise IndexError('pop from emoty queue')
  
    return self.__items.popleft()

  def is_not_empty(self) -> bool:
    return bool(self.__items)
  
  def __repr__(self) -> str:
    return str(self.__items)

  def __len__(self) -> int:
    return len(self.__items)

In [3]:
class Node():
  def __init__(self, state, action = None):
    self.state = state
    self.parent = None
    self.action = action
    self.children = []
    self.path_cost = 0
    self.path_to_goal = [action]
    self.search_depth = 0

  def add_child(self, node):
    node.parent = self
    node.search_depth = self.search_depth + 1
    node.path_cost = self.path_cost + 1
    [node.path_to_goal.append(action) for action in self.path_to_goal]
    self.children.append(node)

In [4]:
def print_matrix(matrix):
  for row in matrix:
    print(row)

def tuple_transform(matrix):
  matrix_tuple = tuple(tuple(row) for row in matrix)
  return matrix_tuple

def searchElement(matrix, x):
  for i, row in enumerate(matrix):
    for j, element in enumerate(row):
      if x == element:
        return i, j
  return None

def compare_matrices(matrix1, matrix2):
    if len(matrix1) != len(matrix2) or len(matrix1[0]) != len(matrix2[0]):
        return False

    for i in range(len(matrix1)):
        for j in range(len(matrix1[0])):
            if matrix1[i][j] != matrix2[i][j]:
                return False

    return True

def moveRight(elementIndex, matrix):
  matrix[elementIndex[0]][elementIndex[1]], matrix[elementIndex[0]][elementIndex[1]+1] = matrix[elementIndex[0]][elementIndex[1]+1], matrix[elementIndex[0]][elementIndex[1]]
  return matrix

def moveLeft(elementIndex, matrix):
  matrix[elementIndex[0]][elementIndex[1]], matrix[elementIndex[0]][elementIndex[1]-1] = matrix[elementIndex[0]][elementIndex[1]-1], matrix[elementIndex[0]][elementIndex[1]]
  return matrix

def moveUp(elementIndex, matrix):
  matrix[elementIndex[0]][elementIndex[1]], matrix[elementIndex[0]-1][elementIndex[1]] = matrix[elementIndex[0]-1][elementIndex[1]], matrix[elementIndex[0]][elementIndex[1]]
  return matrix

def moveDown(elementIndex, matrix):
  matrix[elementIndex[0]][elementIndex[1]], matrix[elementIndex[0]+1][elementIndex[1]] = matrix[elementIndex[0]+1][elementIndex[1]], matrix[elementIndex[0]][elementIndex[1]]
  return matrix

In [5]:
def getPossibleStates(node):
  notMove = node.action
  allMoves = {
      (0,0): ["Right", "Down"],
      (0,1): ["Right", "Down", "Left"],
      (0,2): ["Left", "Down"],
      (1,0): ["Up", "Right", "Down"],
      (1,1): ["Left", "Up", "Right", "Down"],
      (1,2): ["Up", "Left", "Down"],
      (2,0): ["Up", "Right"],
      (2,1): ["Left", "Up", "Right"],
      (2,2): ["Left", "Up"]
  }

  possibleStates = []

 
  emptyIndex = searchElement(node.state, 0)
  possibleMoves = allMoves[emptyIndex]

  for move in possibleMoves:
    state_copy = copy.deepcopy(node.state)
    if move == "Right":
      newState = moveRight(emptyIndex, state_copy)
      possibleStates.append((newState, move))
    elif move == "Left":
      newState = moveLeft(emptyIndex, state_copy)
      possibleStates.append((newState, move))
    elif move == "Up":
      newState = moveUp(emptyIndex, state_copy)
      possibleStates.append((newState, move))
    elif move == "Down":
      newState = moveDown(emptyIndex, state_copy)
      possibleStates.append((newState, move))

  return possibleStates

In [6]:
def search_status(node, expansion, fringe, max_fringe, running_time, max_ram_usage):
  #print(resposta.state)
  path_to_goal = node.path_to_goal[::-1]
  path_to_goal.pop(0)
  print("path_to_goal:", path_to_goal)
  print("cost_of_path:", node.path_cost)
  print("expanded_nodes:", expansion)
  print("fringe_size:", fringe)
  print("max_fringe_size:", max_fringe)
  print("search_depth:", node.path_cost)
  print("running_time:", running_time)
  print("max_ram_usage:", max_ram_usage)

# **Busca em Largura**

In [7]:
def bfs(problem):
  goal_state = [
    [0, 1, 2],
    [3, 4, 5],
    [6, 7, 8]
]
  expanded_nodes = 0
  fringe_size = 0
  max_fringe_size = 0
  
  root_node = Node(problem, None)
  if compare_matrices(root_node.state, goal_state):
    return root_node.path_to_goal, expanded_nodes, fringe_size, max_fringe_size


  frontier = Queue()

  frontier.enqueue(root_node)
  fringe_size += 1
  max_fringe_size += 1
  
  explored = set(tuple_transform(root_node.state))

  while frontier.is_not_empty():
    node = frontier.dequeue()
    if tuple_transform(node.state) not in explored:
      expanded_nodes += 1
    fringe_size -= 1
    state_tuple = tuple_transform(node.state)
    explored.add(state_tuple) 
    
    for state in getPossibleStates(node):
      newNode = Node(state[0], state[1])
      node.add_child(newNode)

      if tuple_transform(newNode.state) not in explored:
        if compare_matrices(newNode.state, goal_state):
          return newNode, expanded_nodes, fringe_size, max_fringe_size
        frontier.enqueue(newNode)
        fringe_size += 1
        max_fringe_size += 1
  return None

process = psutil.Process()
memory_before = process.memory_info().rss
time_before = time.time()

answer = bfs([[0, 8, 7], [6, 5, 4], [3, 2, 1]])

time_after = time.time()
total_time = time_after - time_before
memory_after = process.memory_info().rss
memory = memory_after - memory_before
memory = round(memory / (1024 * 1024), 2)

search_status(answer[0], answer[1], answer[2], answer[3], total_time, memory)

path_to_goal: ['Right', 'Right', 'Down', 'Down', 'Left', 'Up', 'Left', 'Up', 'Right', 'Right', 'Down', 'Down', 'Left', 'Up', 'Left', 'Down', 'Right', 'Up', 'Up', 'Left', 'Down', 'Right', 'Up', 'Right', 'Down', 'Down', 'Left', 'Up', 'Left', 'Up']
cost_of_path: 30
expanded_nodes: 180914
fringe_size: 9021
max_fringe_size: 497374
search_depth: 30
running_time: 35.25674390792847
max_ram_usage: 1232.37


# **Busca em Profundidade Limitada**


In [8]:
def dfs(problem, limit):
  goal_state = [
    [0, 1, 2],
    [3, 4, 5],
    [6, 7, 8]
]
  expanded_nodes = 0
  fringe_size = 0
  max_fringe_size = 0
  
  root_node = Node(problem, None)

  if compare_matrices(root_node.state, goal_state):
    return root_node.path_to_goal

  frontier = []

  frontier.append(root_node)
  fringe_size += 1
  max_fringe_size += 1
  
  explored = set(tuple_transform(root_node.state))

  while frontier:
    node = frontier.pop()

    if node.search_depth == limit:
      continue

    if tuple_transform(node.state) not in explored:
      expanded_nodes += 1
    fringe_size -= 1
    explored.add(tuple_transform(node.state)) 
    
    for state in getPossibleStates(node):      
      newNode = Node(state[0], state[1])
      node.add_child(newNode)

      if tuple_transform(newNode.state) not in explored:
        if compare_matrices(newNode.state, goal_state):
          return newNode, expanded_nodes, fringe_size, max_fringe_size
        frontier.append(newNode)
        fringe_size += 1
        max_fringe_size += 1
  return None

process = psutil.Process()
memory_before = process.memory_info().rss
initial_time = time.time()

answer = dfs([[0, 8, 7], [6, 5, 4], [3, 2, 1]], 40)

final_time = time.time()
total_time = final_time - initial_time
memory_after = process.memory_info().rss
memory = memory_after - memory_before
memory = round(memory / (1024 * 1024), 2)

search_status(answer[0], answer[1], answer[2], answer[3], total_time, memory)

path_to_goal: ['Down', 'Down', 'Right', 'Right', 'Up', 'Left', 'Down', 'Left', 'Up', 'Right', 'Down', 'Right', 'Up', 'Up', 'Left', 'Down', 'Down', 'Right', 'Up', 'Up', 'Left', 'Down', 'Down', 'Right', 'Up', 'Left', 'Up', 'Left', 'Down', 'Down', 'Right', 'Up', 'Up', 'Left', 'Down', 'Right', 'Down', 'Left', 'Up', 'Up']
cost_of_path: 40
expanded_nodes: 36608
fringe_size: 25619
max_fringe_size: 62361
search_depth: 40
running_time: 1.4913074970245361
max_ram_usage: 71.93


# **Busca de Aprofundamento Iterativo em Profundidade**

In [9]:
def idfs(problem, max_depth):
  for limit in range(0, max_depth):
    result = dfs(problem, limit)
    if result is not None:
      return result

process = psutil.Process()
memory_before = process.memory_info().rss
time_before = time.time()

answer = idfs([[0, 8, 7], [6, 5, 4], [3, 2, 1]], 42)

time_after = time.time()
total_time = time_after - time_before
memory_after = process.memory_info().rss
memory = memory_after - memory_before
memory = round(memory / (1024 * 1024), 2)

search_status(answer[0], answer[1], answer[2], answer[3], total_time, memory)

path_to_goal: ['Down', 'Down', 'Right', 'Right', 'Up', 'Left', 'Down', 'Left', 'Up', 'Right', 'Down', 'Right', 'Up', 'Up', 'Left', 'Down', 'Down', 'Right', 'Up', 'Up', 'Left', 'Down', 'Down', 'Right', 'Up', 'Left', 'Up', 'Left', 'Down', 'Down', 'Right', 'Up', 'Up', 'Left', 'Down', 'Right', 'Down', 'Left', 'Up', 'Up']
cost_of_path: 40
expanded_nodes: 36608
fringe_size: 25619
max_fringe_size: 62361
search_depth: 40
running_time: 81.25530791282654
max_ram_usage: -11.75
